In [ ]:
# Import Libraries
import pandas as pd
import pickle as pk
import numpy as np
from collections import defaultdict
import time
import math 
import random

# Open files created in all the previous *.ipynb files

infile = open("Stored Data/aspect_dict.pickle", "rb")
aspect_dict = pk.load(infile)
infile.close()

infile = open("Stored Data/city_dict.pickle", "rb")
city_dict = pk.load(infile)
infile.close()

infile = open("Stored Data/hotel_dict.pickle", "rb")
hotel_dict = pk.load(infile)
infile.close()

infile = open("Stored Data/timeline_dict.pickle", "rb")
timeline_dict = pk.load(infile)
infile.close()

infile = open("Stored Data/best_reviews_dict.pickle", "rb")
best_reviews_dict = pk.load(infile)
infile.close()

# given an array of numbers and a min and max limit, it 
# scales the numbers in the array according to the min
# and max limit specified
def min_max(arr, min_limit, max_limit):
    _min = math.inf
    _max = -math.inf

    for i in range(len(arr)):
        if arr[i] < _min:
            _min = arr[i]

        if arr[i] > _max:
            _max = arr[i]
            
    if _min == _max:
        return [(min_limit+max_limit)/2] * len(arr)

    new_arr = []
    for i in range(len(arr)):
        new_val = (((arr[i] - _min) / (_max - _min)) * (max_limit - (min_limit))) + min_limit
        new_arr.append(new_val)

    return new_arr

"""
The following function is used for normalization purposes. It takes a user-inputted aspect
and user-inputted location as argument. It first gets all the hotels in the location and 
having the aspect along with the sentiment scores of each hotel for that aspect. The scores
are put into an array and are scaled appropriately. The hotels which had negative sentiment 
for that aspect are discarded as the user is only interested in the positive reviews. At
last, it returns a list of tuples which are sorted with the top hotel for the aspect being 
at the top and the other hotels in a descending priority.

"""
def normalize_data(search, location):
    count = 0
    collection = []
    for key, value in timeline_dict.items():
        if count == 4:
            break
        else:
            if location in timeline_dict[key]:
                if search in timeline_dict[key][location]:
                    data = timeline_dict[key][location][search]
                    collection.append(data)
                    count += 1

    results_aspects = {}

    for x in collection:
        for y in x:
            if y[0] in results_aspects:
                results_aspects[y[0]] += y[1]
            else:
                results_aspects.setdefault(y[0], y[1])
                
    pos_aspects = {}

    for key, value in results_aspects.items():
        if value >= 0 :
            pos_aspects.setdefault(key, value)
    
    array_of_num = []

    for key, value in pos_aspects.items():
        array_of_num.append(value)

    normalize_num = min_max(array_of_num,0,100)

    count = 0
    normalize_array = []
    for key, value in pos_aspects.items():
        if normalize_num[count] > 0:
            normalize_array.append((normalize_num[count], key))
        count += 1

    normalize_array.sort(reverse=True)

    return normalize_array    

# for a given hotel, it returns its coordinates, which are used in the 
# leaflet.js API which shows the hotel's location on the map
def get_location(data, hotel_info):
    location = []
    for a in data:
        location.append([a[1], hotel_info[a[1]][0], hotel_info[a[1]][2], hotel_info[a[1]][3]])
        
    return location

timeline = normalize_data('room', 'London, United Kingdom')
# timeline = normalize_data('services', 'Amsterdam, Netherlands')
timeline